In [11]:
import torch
import numpy as np
import pandas as pd
#import logging
import numpy as np
from small_text import TextDataset
from transformers import AutoTokenizer
from small_text.integrations.transformers.classifiers.setfit import SetFitModelArguments
from small_text.integrations.transformers.classifiers.factories import SetFitClassificationFactory
from small_text import (
PoolBasedActiveLearner,
PredictionEntropy,
TransformerBasedClassificationFactory,
TransformerModelArguments,
random_initialization_balanced,
TransformersDataset
)

import gc
import torch
from sklearn.metrics import accuracy_score

In [ ]:
df_train = pd.read_csv('data_stratified_sampling/train_data_stratified.csv')

df_train.head(20)

df_test = pd.read_csv('data_stratified_sampling/test_set_stratified.csv')

In [8]:
num_classes = np.unique(df_train['marginal_text']).shape[0]

In [5]:
from transformers import AutoTokenizer

transformer_model_name = 'KB/bert-base-swedish-cased'

tokenizer = AutoTokenizer.from_pretrained(transformer_model_name)

In [10]:
from small_text import TransformersDataset

target_labels = np.arange(num_classes)

train = TransformersDataset.from_arrays(df_train['text_line'] , df_train['marginal_text'] , tokenizer , max_length = 60 , target_labels=target_labels)

test = TransformersDataset.from_arrays(df_test['text_line'] , df_test['marginal_text'], tokenizer, max_length = 60 , target_labels=target_labels)

/home/drunken-monkey/miniconda3/envs/swerik/lib/python3.9/site-packages/small_text/utils/annotations.py:67: ExperimentalWarning: The function from_arrays is experimental and maybe subject to change soon.
  warnings.warn(f'The {subject} {func_or_class.__name__} is experimental '


In [13]:
train.y

array([0, 0, 0, ..., 0, 0, 0])

In [17]:
def intialize_active_learner(active_learner , y_train):

    indicies_intial = random_initialization_balanced(y_train , n_samples = 20)

    active_learner.initialize_data(indicies_intial, y_train[indicies_intial])

    return indicies_intial

transformer_model = TransformerModelArguments(transformer_model_name)

clf_factory = TransformerBasedClassificationFactory(transformer_model , num_classes, kwargs=dict({'device':'cuda', 'mini_batch_size':32, 'class_weight':'balanced'}))


query_startegy = PredictionEntropy()

active_learner = PoolBasedActiveLearner(clf_factory , query_startegy, train)

indices_labeled = intialize_active_learner(active_learner,train.y)
    

/home/drunken-monkey/miniconda3/envs/swerik/lib/python3.9/site-packages/small_text/integrations/pytorch/classifiers/base.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

In [18]:
from scipy.stats import entropy
from small_text.utils.data import list_length

num_queries = 5

def get_entropy_labelled(row, model, indices_labeled):
    p = model.predict_proba(row)
    entr = np.apply_along_axis(lambda x: entropy(x) ,1 ,p)
    query2 = pd.DataFrame(entr , indices_labeled)
    query3 = np.array(query2.nlargest(1,0).index.values, dtype=np.int64)

    return query3

def evaluate(active_learner , train, test, indices_labeled):
    y_pred = active_learner.classifier.predict(train)

    y_prob = get_entropy_labelled(train , active_learner.classifier, indices_labeled)

    y_pred_test = active_learner.classifier.predict(test)

    test_acc = accuracy_score(y_pred_test , test.y)

    print(f'Train accuracy {accuracy_score(y_pred , train.y)}')
    print(f'Test accuracy {test_acc}')

    return test_acc, y_prob

results = []
eval, mix_query = evaluate(active_learner , train[indices_labeled] , test, indices_labeled)
results.append(eval)



Train accuracy 0.9
Test accuracy 0.8346153846153846


In [20]:
for i in range(num_queries):
    indices_queried = active_learner.query(num_samples=5)

    y = train.y[indices_queried]

    yy = train.y[mix_query]

    active_learner.update(y)

    active_learner.update_label_at(mix_query[0],yy[0],retrain = True)

    indicies_label = np.concatenate([indices_queried , indices_labeled])

    print('---------------')
    print(f'Iteration #{i} ({len(indices_labeled)} samples)')
    eval, mix_query = evaluate(active_learner, train[indices_labeled], test,indices_labeled)
    results.append(eval)


/home/drunken-monkey/miniconda3/envs/swerik/lib/python3.9/site-packages/small_text/integrations/pytorch/classifiers/base.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

---------------
Iteration #0 (20 samples)
Train accuracy 0.9
Test accuracy 0.9038461538461539


/home/drunken-monkey/miniconda3/envs/swerik/lib/python3.9/site-packages/small_text/integrations/pytorch/classifiers/base.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

---------------
Iteration #1 (20 samples)
Train accuracy 0.9
Test accuracy 0.801923076923077


/home/drunken-monkey/miniconda3/envs/swerik/lib/python3.9/site-packages/small_text/integrations/pytorch/classifiers/base.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

---------------
Iteration #2 (20 samples)
Train accuracy 0.95
Test accuracy 0.9115384615384615


/home/drunken-monkey/miniconda3/envs/swerik/lib/python3.9/site-packages/small_text/integrations/pytorch/classifiers/base.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

---------------
Iteration #3 (20 samples)
Train accuracy 0.9
Test accuracy 0.9038461538461539


/home/drunken-monkey/miniconda3/envs/swerik/lib/python3.9/site-packages/small_text/integrations/pytorch/classifiers/base.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

---------------
Iteration #4 (20 samples)
Train accuracy 1.0
Test accuracy 0.9076923076923077
